In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table, ctx
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the CRUD operations class for MongoDB
from MongoDB_CRUD import CRUD_Operations


###########################
# Data Manipulation / Model
###########################

# Set up the connection parameters for specific database use
username = "aacuser"
password = "catsrule"
host = 'nv-desktop-services.apporto.com'
port = 33410
db_name = 'AAC'
collection_name = 'animals'

# Create an instance of the CRUD_Operations class to handle database operations
db = CRUD_Operations(username, password, host, port, db_name, collection_name)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(
        style={"display": "flex", "justify-content": "center", "align-items": "center"},
        children=[
            html.Img(src="assets/GS-logo.png", style={"height": "60px"}),
            html.B(html.H1('Grazioso Salvare Dashboard - Noelle Bishop', style={"margin": "0"}))]
    ),
    html.Div(className='buttonRow',
            style={'display': 'flex'},
            children=[
                html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                html.Button(id='submit-button-two', n_clicks=0, children='Mountain or Wilderness Rescue'),
                html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue or Individual Tracking'),
                html.Button(id='submit-button-four', n_clicks=0, children='Reset (All Animals)')
                ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    # This element adds a paragraph with some identifier text
    html.P("Brought to you by the development team at Global Rain - 2025"),
    
    html.Div(
        style={"display": "flex", "justify-content": "space-between", "align-items": "center"},
        children= [
            # Pie chart for specialized rescue breeds
            html.Div(id="graph-id", style={"flex": "1"}),
            # Geolocation map for first animal in sorted or unsorted list
            html.Div(id="map-id", style={"flex": "1"})
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'data'),
    [Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two', 'n_clicks'),
     Input('submit-button-three', 'n_clicks'),
     Input('submit-button-four', 'n_clicks')
    ])

def on_click(button1, button2, button3, button4):
    # Read the data from the database and convert it to a DataFrame (df)
    df = pd.DataFrame.from_records(db.read({}))

    # Callback context determines while button was most recently clicked
    # https://dash.plotly.com/advanced-callbacks?from_column=20423&from=20423
    if not ctx.triggered_id:
        # If no button has been clicked, return the full dataset
        df = pd.DataFrame.from_records(db.read({}))
    else:
        # Change filtering based on most recently clicked button
        if ctx.triggered_id == 'submit-button-one':  # Water Rescue
            df = pd.DataFrame.from_records(db.read({
                                                "breed": {"$in": ["Labrador Retriever Mix", "Chesa Bay Retr Mix", "Newfoundland"]},
                                                "sex_upon_outcome": "Intact Female",
                                                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
                                                }))
        elif ctx.triggered_id == 'submit-button-two':  # Mountain or Wilderness Rescue
            df = pd.DataFrame.from_records(db.read({
                                                "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                                                "sex_upon_outcome": "Intact Male",
                                                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
                                                }))
        elif ctx.triggered_id == 'submit-button-three':  # Disaster Rescue or Individual Tracking
            df = pd.DataFrame.from_records(db.read({
                                                "breed": {"$in": ["German Shepherd", "Doberman Pinscher", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                                                "sex_upon_outcome": "Intact Male",
                                                "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
                                                }))
        elif ctx.triggered_id == 'submit-button-four':  # Reset (All Animals)
            df = pd.DataFrame.from_records(db.read({}))

    # Clean up Mongo _id field
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    return df.to_dict('records')

# This callback updates the style of the selected columns in the table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of a list. 
# For this application, we are only permitting single row selection so there is only one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):  
    if not viewData or len(viewData) == 0:
        return [html.P("No data available to map.")]

    dff = pd.DataFrame.from_dict(viewData)

    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None or len(index) == 0:
        row = 0
    else: 
        row = index[0]

    # Austin, TX (location of shelter) is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
            center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup:
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# This callback creates a pie chart based on the selected data
# Pie chart guidance was found in another one of Prof. Steve Satterfield's videos: https://www.youtube.com/watch?v=sEXgZ7c12Q8
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')])

def update_graphs(viewData):
    if not viewData or len(viewData) == 0:
        return []

    df = pd.DataFrame.from_records(viewData)
    return dcc.Graph(figure = px.pie(df, names='breed', title = 'Specialized Rescue Breeds'))

# Run the Dash app!
app.run_server(debug=True)